# Building and plotting a Confusion Matrix

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)      # Already on by default

def get_num_correct(preds, labels):       # 预测正确的数量
    return preds.argmax(dim=1).eq(labels).sum().item()

class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=12, kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=12*4*4, out_features=120)     # Linear = fully connected(fc) = dense
        self.fc2 = nn.Linear(in_features=120, out_features=60)
        self.out = nn.Linear(in_features=60, out_features=10)
        
    def forward(self, t):
        # (1) input layer
        t = t
        
        # (2) hidden conv layer
        t = self.conv1(t)
        t = F.relu(t)                   # 卷积层中不包含激活函数，需要手动添加
        t = F.max_pool2d(t, kernel_size=2, stride=2)     # 池化操作（没有权重的函数不能称为层）
        
        # (3) hidden conv layer
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t, kernel_size=2, stride=2)
        
        # (4) hidden linear layer
        t = t.reshape(-1, 12*4*4)       # 必须手动flatten
        t = self.fc1(t)
        t = F.relu(t)
        
        # (5) hidden linear layer
        t = self.fc2(t)
        t = F.relu(t)
        
        # (6) output layer
        t = self.out(t)
        # t = F.softmax(t, dim=1)           # 不直接在forward中用softmax，而是在训练过程中用cross-entropy损失函数计算loss，其中自带softmax
        
        return t

network = Network()
train_set = torchvision.datasets.FashionMNIST(
    root = './data/FashionMNIST',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor()
    ])
)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=100)

In [2]:
len(train_set)

60000

In [3]:
len(train_set.targets)

60000

## Getting predictions for the entire training set

In [18]:
def get_all_preds(model, loader):
    all_preds = torch.tensor([])
    for batch in loader:              # 按batch进行处理
        images, labels = batch        # 提取图像和标签
        
        preds = model(images)         # 得到当前batch的预测标签
        all_preds = torch.cat(        # 将每批预测标签添加进去
            (all_preds, preds),
            dim=0)
        
    return all_preds

In [5]:
prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
train_preds = get_all_preds(network, prediction_loader)

In [6]:
train_preds.shape

torch.Size([10000, 10])

In [8]:
print(train_preds.requires_grad)      # 自动计算梯度打开会增加开销

True


In [9]:
train_preds.grad

C:\Users\Ganzeus\AppData\Local\Temp\ipykernel_4240\3308146931.py:1: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at C:\cb\pytorch_1000000000000\work\build\aten\src\ATen/core/TensorBody.h:485.)
  train_preds.grad


In [10]:
train_preds.grad_fn           # 计算梯度的函数

In [19]:
with torch.no_grad():         # 关闭梯度追踪
    prediction_loader = torch.utils.data.DataLoader(train_set, batch_size=10000)
    train_preds = get_all_preds(network, prediction_loader)

In [12]:
print(train_preds.requires_grad) 

False


In [20]:
train_preds.grad

In [14]:
train_preds.grad_fn

In [22]:
preds_correct = get_num_correct(train_preds, train_set.targets)

print("total correct:", preds_correct)
print("accuracy:", preds_correct / len(train_set))

total correct: 6000
accuracy: 0.1


In [21]:
train_preds.shape

torch.Size([60000, 10])